# Проект классификации отзывов для интернет-магазина

 Магазину требуется инструмент, который будет искать токсичные комментарии, оставляемые пользователями в описании товаров магазина и отправлять их на модерацию.

**Цель проекта:** Обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Целевое значение метрики качества *F1* не меньше 0.75.


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка данных

In [ ]:
!pip install spacy -q

In [ ]:
import re
import pandas as pd
import spacy
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,  GridSearchCV

In [ ]:
df_comm = pd.read_csv("/datasets/toxic_comments.csv")

In [ ]:
RANDOM_STATE=12345

In [ ]:
df_comm.head(10)

Unnamed: 0                                               text  toxic
0           0  Explanation\nWhy the edits made under my usern...      0
1           1  D'aww! He matches this background colour I'm s...      0
2           2  Hey man, I'm really not trying to edit war. It...      0
3           3  "\nMore\nI can't make any real suggestions on ...      0
4           4  You, sir, are my hero. Any chance you remember...      0
5           5  "\n\nCongratulations from me as well, use the ...      0
6           6       COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK      1
7           7  Your vandalism to the Matt Shirvington article...      0
8           8  Sorry if the word 'nonsense' was offensive to ...      0
9           9  alignment on this subject and which are contra...      0

In [ ]:
df_comm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [ ]:
df_comm.duplicated().sum()

0

In [ ]:
df_comm.isna().sum()

Unnamed: 0    0
text          0
toxic         0
dtype: int64

Датасет состоит из 159292 строк, столбца с текстами сообщений и стобца с разметкой целевого признака. Пропусков и дубликатов в датасете нет. Посмотрим на соотношение классов целевого признака

In [ ]:
df_comm['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Имеет место выраженный дисбаланс целевого признака (нетоксичных комментариев почти в 10 раз больше чем токсичных), однако менять датасет не будем чтобы получить более честную работу моделей.

 Для подготовки данных к обучению удалим лишние символы и пробелы с помощью регулярных выражений, токенизируем и лемматизируем текст. Очищенный текст добавим в датасет. Для удобства выполним подготовку с помощью функции.

In [ ]:
def clean_text(text):

    text = text.lower()
    regular = r'[\*+\#+\№\"\-+\+\=+\?+\&\^\.+\;\,+\>+\(\)\/+\:\\+]'
    regular_url = r'(http\S+)|(www\S+)|([\w\d]+www\S+)|([\w\d]+http\S+)'
    text = re.sub(regular, '', text)
    text = re.sub(r'(\d+\s\d+)|(\d+)',' NUM ', text)
    text = re.sub(r'\s+', ' ', text)
    return text


nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
def lemmatize_text(text):
    text = clean_text(text)
    doc = nlp(text)
    text = [token.lemma_ for token in doc]
    text = ' '.join(text)
    return text

tqdm.pandas()
df_comm['lemm_text'] = df_comm['text'].progress_apply(lemmatize_text)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [ ]:
df_comm.head()

Unnamed: 0                                               text  toxic  \
0           0  Explanation\nWhy the edits made under my usern...      0   
1           1  D'aww! He matches this background colour I'm s...      0   
2           2  Hey man, I'm really not trying to edit war. It...      0   
3           3  "\nMore\nI can't make any real suggestions on ...      0   
4           4  You, sir, are my hero. Any chance you remember...      0   

                                           lemm_text  
0  explanation why the edit make under my usernam...  
1  d'aww ! he match this background colour I be s...  
2  hey man I be really not try to edit war it be ...  
3    more I can not make any real suggestion on i...  
4  you sir be my hero any chance you remember wha...

 Следующим этапом выделим признаки, на которых будем обучать модели. Разделим датасет на тренировочную и тествовую выборки, при разделении учтем дисбаланс классов.

In [ ]:
X=df_comm['lemm_text']
y=df_comm['toxic']

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, stratify=y, random_state=12345)

## Обучение

для обучения моделей используем пайплайн с классификаторами - дерево решений, логистическая регрессия, KNN. Проведем кросс-валидацию для подбора гиперпараметров моделей. Результаты обучения всех моделей выведем в таблицу.

In [ ]:
pipe = Pipeline([
    ('vect', TfidfVectorizer()),
    ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))
])

In [ ]:
param_grid = [
    {
    "vect__ngram_range": ((1, 1), (1, 2))
    },
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        'models__min_samples_leaf': range(3,6),
        'models__max_depth': range(3,6)
    },
    {
        'models': [LogisticRegression(random_state=RANDOM_STATE)],
        'models__C': range(5,15),
        'models__penalty': ['l1', 'l2', 'elasticnet']
    } ,
    {
        'models': [KNeighborsClassifier(n_neighbors=300)]

    }
                 ]

In [ ]:
grid_search = GridSearchCV(
    pipe,
    param_grid=param_grid,
    cv=3,
    scoring='f1',
    n_jobs=-1
)

In [ ]:
grid_search.fit(X_train, y_train)

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  warni

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect', TfidfVectorizer()),
                                       ('models',
                                        DecisionTreeClassifier(random_state=12345))]),
             n_jobs=-1,
             param_grid=[{'vect__ngram_range': ((1, 1), (1, 2))},
                         {'models': [DecisionTreeClassifier(random_state=12345)],
                          'models__max_depth': range(3, 6),
                          'models__min_samples_leaf': range(3, 6)},
                         {'models': [LogisticRegression(C=13,
                                                        random_state=12345)],
                          'models__C': range(5, 15),
                          'models__penalty': ['l1', 'l2', 'elasticnet']},
                         {'models': [KNeighborsClassifier(n_neighbors=300)]}],
             scoring='f1')

In [ ]:
res = pd.DataFrame(grid_search.cv_results_)
res = res.drop (['std_fit_time','mean_score_time',
                 'std_score_time','params','split0_test_score','split1_test_score',
                 'split2_test_score','std_test_score'], axis=1)
res

mean_fit_time param_vect__ngram_range  \
0      133.197824                  (1, 1)   
1      604.631311                  (1, 2)   
2        4.764830                     NaN   
3        4.720769                     NaN   
4        4.699417                     NaN   
5        5.151276                     NaN   
6        5.111781                     NaN   
7        5.119505                     NaN   
8        5.577193                     NaN   
9        5.585274                     NaN   
10       5.579880                     NaN   
11       3.504694                     NaN   
12      44.562241                     NaN   
13       3.352815                     NaN   
14       3.470248                     NaN   
15      45.621843                     NaN   
16       3.474125                     NaN   
17       3.407991                     NaN   
18      44.688836                     NaN   
19       3.445324                     NaN   
20       3.570621                     NaN   
21      44.466175                     NaN   
22       3.450670                     NaN   
23       3.397793                     NaN   
24      45.913049                     NaN   
25       3.461948                     NaN   
26       3.540653                     NaN   
27      45.443283                     NaN   
28       3.505873                     NaN   
29       3.486854                     NaN   
30      44.525688                     NaN   
31       3.392231                     NaN   
32       3.472540                     NaN   
33      44.616034                     NaN   
34       3.486259                     NaN   
35       3.447714                     NaN   
36      45.851809                     NaN   
37       3.441326                     NaN   
38       3.433419                     NaN   
39      45.048609                     NaN   
40       3.419187                     NaN   
41       3.420741                     NaN   

                                    param_models param_models__max_depth  \
0                                            NaN                     NaN   
1                                            NaN                     NaN   
2     DecisionTreeClassifier(random_state=12345)                       3   
3     DecisionTreeClassifier(random_state=12345)                       3   
4     DecisionTreeClassifier(random_state=12345)                       3   
5     DecisionTreeClassifier(random_state=12345)                       4   
6     DecisionTreeClassifier(random_state=12345)                       4   
7     DecisionTreeClassifier(random_state=12345)                       4   
8     DecisionTreeClassifier(random_state=12345)                       5   
9     DecisionTreeClassifier(random_state=12345)                       5   
10    DecisionTreeClassifier(random_state=12345)                       5   
11  LogisticRegression(C=13, random_state=12345)                     NaN   
12  LogisticRegression(C=13, random_state=12345)                     NaN   
13  LogisticRegression(C=13, random_state=12345)                     NaN   
14  LogisticRegression(C=13, random_state=12345)                     NaN   
15  LogisticRegression(C=13, random_state=12345)                     NaN   
16  LogisticRegression(C=13, random_state=12345)                     NaN   
17  LogisticRegression(C=13, random_state=12345)                     NaN   
18  LogisticRegression(C=13, random_state=12345)                     NaN   
19  LogisticRegression(C=13, random_state=12345)                     NaN   
20  LogisticRegression(C=13, random_state=12345)                     NaN   
21  LogisticRegression(C=13, random_state=12345)                     NaN   
22  LogisticRegression(C=13, random_state=12345)                     NaN   
23  LogisticRegression(C=13, random_state=12345)                     NaN   
24  LogisticRegression(C=13, random_state=12345)                     NaN   
25  LogisticRegression(C=13, random_state=12345)                     NaN   
26  LogisticRegr

In [ ]:
print('The best model and its parameters:\n\n', grid_search.best_estimator_)
print ('The metric of the best model on cross-validation:', grid_search.best_score_)

The best model and its parameters:

 Pipeline(steps=[('vect', TfidfVectorizer()),
                ('models', LogisticRegression(C=13, random_state=12345))])
The metric of the best model on cross-validation: 0.7812920327938802


In [ ]:
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred_test = grid_search.best_estimator_.predict(X_test)
f1_test = f1_score(y_test, y_pred_test)

print ("F1 on test set is:", f1_test)

F1 on test set is: 0.7850360370666971


Лучшая модель, выбранная при кросс-валидации - логистическая регрессия с параметром регуляризации capacity =13, регуляризацией l2, значение метрики F1 на кросс-валидации 78.1%, на тестовой выбоке  78,5% (удлвлетворяют требованиям проекта).

## Выводы

Согласно цели проекта - обучить модель выявлять токсичные комментарии на сайте магазина была проведена загрузка необходимых библиотек, датасета, проведена первичная оценка данных.

На втором этапе проведена проведена подготовка текстов комментариев к обучению моделей -  удалены лишние символы, пробелы,  проведена токенизация и лемматизация текста.  

Лемматизированный текст передан пайплайну, включающему векторайзер TF-IDF, 3 модели (логистическая регрессия, дерево решений, метод ближайших соседей).

Проведена кросс-валидация для подбора лучшей модели и ее гиперпараметров, лучшей моделью выбрана логистическая регрессия, метрика F1 на кросс-валидации 78,1%. Получены предсказания на тествой выборке, метрика F1 на тествовой выборке составила 78,5%.

Таким образом, логистическая регрессия с учетом ее быстрой работы и удовлетворительного качества предсказаний может быть использована для отбора токсичных комментариев.